# Profiling Julia code

In [1]:
using Base.Profile

In [12]:
function myfunc()
           A = rand(1000, 1000)
           a = maximum(A)
           r = A*A
           return r
       end

myfunc();

In [71]:
Profile.clear()
Profile.init(n = 10^5, delay = 0.00001)

In [72]:
@profile myfunc();

In [73]:
Profile.print()

857 ./task.jl:335; (::IJulia.##14#17)()
 857 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  857 .../Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1}...
   856 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::...
    856 .../Compat/src/Compat.jl:71; include_string(::Module, ::Strin...
     856 ./loading.jl:522; include_string(::String, ::String)
      856 ./<missing>:?; anonymous
       856 ./profile.jl:23; macro expansion
        550 ./In[12]:2; myfunc()
         393 ./random.jl:452; rand!(::MersenneTwister, ::A...
          393 ./dSFMT.jl:84; dsfmt_fill_array_close_open!...
         157 ./random.jl:285; rand
          157 ./random.jl:284; rand
           157 ./random.jl:387; rand
            157 ./random.jl:390; rand
             1 ....6/ZMQ/src/ZMQ.jl:418; close(::ZMQ.Message)
        198 ./In[12]:3; myfunc()
         198 ./reduce.jl:280; _mapreduce(::Base.#identity,...
          40  ./reduce.jl:430; mapreduce_impl(::Base.#iden...
          19  ./reduce

#### `combine` keyword argument

Some lines of code contain multiple operations; for example, `s += A[i]` contains both an array reference (`A[i]`) and a sum operation. These correspond to different lines in the generated machine code, and hence there may be two or more different addresses captured during backtraces on this line. **`combine = true`** lumps them together, and is probably what you typically want, but you can generate an output separately for each unique instruction pointer with **`combine = false`**.

In [16]:
#Profile.print(format=:flat)
Profile.print(combine=true)

6 ./task.jl:335; (::IJulia.##14#17)()
 6 ...IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  6 ...6/Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1},...
   6 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::I...
    6 .../Compat/src/Compat.jl:71; include_string(::Module, ::String...
     6 ./loading.jl:522; include_string(::String, ::String)
      6 ./<missing>:?; anonymous
       6 ./profile.jl:23; macro expansion
        2 ./In[12]:2; myfunc()
         2 ./random.jl:452; rand!(::MersenneTwister, ::Ar...
          2 ./dSFMT.jl:84; dsfmt_fill_array_close_open!(...
        2 ./In[12]:3; myfunc()
         2 ./reduce.jl:280; _mapreduce(::Base.#identity, ...
          2 ./reduce.jl:432; mapreduce_impl(::Base.#identi...
        2 ./In[12]:4; myfunc()
         2 ./linalg/matmul.jl:367; gemm_wrapper!(::Array{Float64...
          2 ./linalg/blas.jl:1027; gemm!(::Char, ::Char, ::Floa...


In [17]:
#Profile.print()

#### `maxdepth` keyword argument

We can specify a `maxdepth` to avoid going displaying information about the function calls that are inside other function calls of more than `maxdepth`.

In [18]:
Profile.print(combine=false, maxdepth=5)

6 ./task.jl:335; (::IJulia.##14#17)()
 6 ...IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  6 ...6/Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1},...
   6 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::I...
    6 .../Compat/src/Compat.jl:71; include_string(::Module, ::String...
     6 ./loading.jl:522; include_string(::String, ::String)


#### `mincount` keyword argument

If the profiler shows "too much irrelevant stuff", for example, lines with 1, we can pass as argument `mincount=2`. This will print only parts of the profile with 2 or more.

In [19]:
Profile.print(combine=false,maxdepth=9, mincount=2)

6 ./task.jl:335; (::IJulia.##14#17)()
 6 ...IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  6 ...6/Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1},...
   6 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::I...
    6 .../Compat/src/Compat.jl:71; include_string(::Module, ::String...
     6 ./loading.jl:522; include_string(::String, ::String)
      6 ./<missing>:?; anonymous
       6 ./profile.jl:23; macro expansion
        2 ./In[12]:2; myfunc()
         2 ./random.jl:452; rand!(::MersenneTwister, ::Ar...
        2 ./In[12]:3; myfunc()
         2 ./reduce.jl:280; _mapreduce(::Base.#identity, ...
        2 ./In[12]:4; myfunc()
         2 ./linalg/matmul.jl:367; gemm_wrapper!(::Array{Float64...


### More examples

In [29]:
using Base.Profile

In [30]:
function examplefunc(x)
    x = x + 2
    x = 2 * x
    return x
end

examplefunc (generic function with 1 method)

In [31]:
aux = rand(10000);

In [32]:
Profile.clear()

In [36]:
@time examplefunc(aux);

  0.000026 seconds (8 allocations: 156.563 KiB)


In [37]:
@profile examplefunc(aux);

In [38]:
Profile.print()

running it multiple times), or adjust the delay between samples with
Profile.init().


In [39]:
@profile (for i = 1:100; examplefunc(aux); end)

In [40]:
Profile.print()

8 ./task.jl:335; (::IJulia.##14#17)()
 8 ...IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  8 ...6/Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1},...
   8 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::I...
    8 .../Compat/src/Compat.jl:71; include_string(::Module, ::String...
     8 ./loading.jl:522; include_string(::String, ::String)
      8 ./<missing>:?; anonymous
       8 ./profile.jl:23; macro expansion
        8 ./In[39]:1; macro expansion
         6 ./In[30]:2; examplefunc
          6 ./arraymath.jl:48; +(::Array{Float64,1}, ::Int64)
           6 ./broadcast.jl:455; broadcast
            6 ./broadcast.jl:316; broadcast_c
             6 ./broadcast.jl:270; broadcast_t
              6 ./broadcast.jl:141; _broadcast!
               6 ./broadcast.jl:149; macro expansion
                6 ./simdloop.jl:73; macro expansion
                 6 ./broadcast.jl:155; macro expansion
         2 ./In[30]:3; examplefunc
          2 ./arraymath.jl:45; *(::Int64

In [41]:
Profile.print(C=true)

8 ..._osx64/build/src/task.c:267; start_task
 8 ...64/build/src/./julia.h:1424; jl_apply
   8 ./task.jl:335; (::IJulia.##14#17)()
    8 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
     8 .../build/src/builtins.c:464; jl_f__apply_latest
      8 ...build/src/builtins.c:426; jl_f__apply
       8 ...build/src/./julia.h:1424; jl_apply
        8 ...ompat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,...
         8 .../execute_request.jl:158; execute_request(::ZMQ.Socket,...
           8 ...ompat/src/Compat.jl:71; include_string(::Module, ::St...
            8 ...ild/src/builtins.c:496; jl_toplevel_eval_in
             8 ...ld/src/toplevel.c:577; jl_toplevel_eval_flex
              8 ...src/interpreter.c:34; jl_interpret_toplevel_expr
               8 ...src/interpreter.c:242; eval
                8 ...src/interpreter.c:75; do_call
                 8 ...b/julia/sys.dylib:?; jlcall_include_string_33515
                  8 ./loading.jl:522; include_string(::String,...
              

#### example 2

In [42]:
Profile.clear()

In [43]:
function testfunc()
    x = rand(10000, 1000)
    y = std(x, 1)
    return y
end

testfunc (generic function with 1 method)

In [44]:
@profile testfunc()

1×1000 Array{Float64,2}:
 0.288813  0.289132  0.290146  0.287452  …  0.288259  0.28713  0.291357

In [48]:
Profile.print(mincount=30)

316 ./task.jl:335; (::IJulia.##14#17)()
 316 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  316 .../Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1}...
   315 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::...
    315 .../Compat/src/Compat.jl:71; include_string(::Module, ::Strin...
     315 ./loading.jl:522; include_string(::String, ::String)
      315 ./<missing>:?; anonymous
       315 ./profile.jl:23; macro expansion
        57  ./In[43]:2; testfunc()
         48 ./random.jl:452; rand!(::MersenneTwister, ::Ar...
          48 ./dSFMT.jl:84; dsfmt_fill_array_close_open!(...
        258 ./In[43]:3; testfunc()
         258 ./<missing>:0; (::Base.#kw##var)(::Array{Any...
          153 ./statistics.jl:204; #var#537(::Bool, ::Void, ::...
           58 ./<missing>:0; (::Base.#kw##varm)(::Array{A...
            58 ./<missing>:0; (::Base.#kw##varm!)(::Array{...
           95 ./statistics.jl:39; mean!(::Array{Float64,2}, ::...
            95 ./<missing>:0; (::B

### ProfileView

In [49]:
using ProfileView

INFO: Recompiling stale cache file /Users/david/.julia/lib/v0.6/ProfileView.ji for module ProfileView.
INFO: Recompiling stale cache file /Users/david/.julia/lib/v0.6/ProfileViewSVG.ji for module ProfileViewSVG.


In [50]:
function profile_test(n)
    for i = 1:n
        A = randn(100,100,20)
        m = maximum(A)
        Afft = fft(A)
        Am = mapslices(sum, A, 2)
        B = A[:,:,5]
        Bsort = mapslices(sort, B, 1)
        b = rand(100)
        C = B.*b
    end
end

profile_test (generic function with 1 method)

In [51]:
profile_test(1)  # run once to trigger compilation
Profile.clear()  # in case we have any previous profiling data
@profile  profile_test(100)